In [1]:
library(Seurat)
library(dplyr)
library(patchwork)
library(ggplot2)
library(gridExtra)
library(stringi)
library(ROCit)
library(png)
library(patchwork)

Warning message:
“程辑包‘Seurat’是用R版本4.2.2 来建造的”
Attaching SeuratObject

Warning message:
“程辑包‘dplyr’是用R版本4.2.3 来建造的”

载入程辑包：‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Warning message:
“程辑包‘patchwork’是用R版本4.2.1 来建造的”
Warning message:
“程辑包‘ggplot2’是用R版本4.2.2 来建造的”
Warning message:
“程辑包‘gridExtra’是用R版本4.2.1 来建造的”

载入程辑包：‘gridExtra’


The following object is masked from ‘package:dplyr’:

    combine


Warning message:
“程辑包‘png’是用R版本4.2.2 来建造的”


In [2]:
load("dat/st.RData")

In [3]:
count<- st@assays$Spatial@counts
count<- (count-min(count))/(max(count)-min(count))
count<-data.frame(count)
count<-t(count)
count<-data.frame(count)
colnames(count)<-rownames(st@assays$Spatial@counts)
rownames(count)<-colnames(st@assays$Spatial@counts)
gene<-rownames(st@assays$Spatial@counts)
head(count[1:5,1:5])
target<-data.frame(st@meta.data[,"TLS_2_cat_annot"])
y<-c()
for(i in target$st.meta.data....TLS_2_cat_annot..){
  if(is.na(i)){
    y<-append(y,0)
  }else if(i=="TLS"){
    y<-append(y,1)
  }else{
    y<-append(y,0)
  }
}
y<-as.data.frame(y)
colnames(y)="y"
rownames(y)<-rownames(target)
count<-cbind(count,y)

Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 1.3 GiB”


,SAMD11,NOC2L,KLHL17,PLEKHN1,HES4
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
c_3_AAACAACGAATAGTTC-1,0,0.0002321802,0.0001160901,0,0.0006965405
c_3_AAACAAGTATCTCCCA-1,0,0.0001160901,0.0000000000,0,0.0000000000
c_3_AAACAATCTACTAGCA-1,0,0.0001160901,0.0000000000,0,0.0001160901
c_3_AAACACCAATAACTGC-1,0,0.0002321802,0.0000000000,0,0.0002321802
c_3_AAACAGAGCGACTCCT-1,0,0.0002321802,0.0000000000,0,0.0001160901


In [4]:
set.seed(123) 
count_positive<-subset(count,count$y==1)
count_negative<-subset(count,count$y==0)
train <- sample(1:nrow(count_positive),nrow(count_positive)*0.02)
count_positive_train <- count_positive[train,]
train <- sample(1:nrow(count_negative),nrow(count_negative)*0.02)
count_negative_train <- count_negative[train,]
count_train<-rbind(count_negative_train,count_positive_train)

In [5]:
remove(st)
remove(count)

In [6]:
library(FSinR)
RFSM_evaluator <- ReliefFeatureSetMeasure(iterations = 1,kNeightbours = 3)
feature<-c()
s=Sys.time()
for(i in gene){
    feature<-append(feature,RFSM_evaluator(count_train,'y',i))
}
e=Sys.time()
print(e-s)

Time difference of 4.722742 hours


In [7]:
length(rownames(count_train))

[1] 235

In [8]:
write.csv(feature,"RI_RELIEF.csv")

In [9]:
names(feature)<-gene

In [11]:
write.csv(feature,"RI_RELIEF_1.csv")